In [ ]:
print('hello world')

In [ ]:
from io import StringIO
import pandas as pd

In [ ]:
TESTDATA = StringIO("""
ID	Link	Name	Backlog	Next	In-progress	Done	CoS	WIT	Deadline	Blocked Days	Deleted	Labels
6		What is the longest you've been awake?	2019-02-06	2019-02-06	2019-02-06	2019-02-07	Intangible	V				
11		What are your favorite hot dog toppings?	2019-02-07	2019-02-07	2019-02-07	2019-02-07	Intangible	V				
17		Where did you get your name?	2019-02-07	2019-02-07	2019-02-07	2019-02-07	Standard	T				
20		Have you ever fainted?	2019-02-07	2019-02-07	2019-02-07	2019-02-07	Standard	D			2019-02-07	
14		What is your favorite lunch meal?	2019-02-06	2019-02-06	2019-02-06	2019-02-08	Expedite	D			2019-02-08	
9		What do you do as excerise?	2019-02-07	2019-02-07	2019-02-07	2019-02-08	Standard	V			2019-02-08	
13		which do you like better, being a passenger or driver?	2019-02-07	2019-02-07	2019-02-08	2019-02-08	Intangible	D				
4		Have you ever bungee jumped?	2019-02-08	2019-02-08	2019-02-08	2019-02-08	Standard	C				
16		Have you ever jumped out of a plane?	2019-02-08	2019-02-08	2019-02-08	2019-02-08	Expedite	L				
15		What are your favorite movies?	2019-02-08	2019-02-08	2019-02-08	2019-02-09	Intangible	C			2019-02-09	
10		Do you like flying?	2019-02-09	2019-02-09	2019-02-09	2019-02-09	Intangible	V				
18		What is your favorite school subject?	2019-02-09	2019-02-09	2019-02-09	2019-02-09	Standard	T				
19		Do you swear often?	2019-02-09	2019-02-09	2019-02-09	2019-02-09	Standard	C				
2		Which do you like better, cars or trucks?	2019-02-09	2019-02-10	2019-02-10	2019-02-10	Standard	D	2019-02-11			
3		Are you religious?	2019-02-09	2019-02-10	2019-02-10	2019-02-10	Standard	D				
5		Do you have nice handwriting?	2019-02-10	2019-02-10	2019-02-10	2019-02-10	Standard	V				
8		Have you ever been on TV?	2019-02-07	2019-02-07	2019-02-08	2019-02-17	Standard	C		6		
12		Have you ever burnt your hair?	2019-02-06	2019-02-06	2019-02-08	2019-02-22	Standard	V		15		
7		Have you ever ironed clothes?	2019-02-08	2019-02-08	2019-02-22	2019-02-22	Intangible	V	2019-02-08			
1		What is your favorite dinner meal?	2019-02-07	2019-02-22	2019-02-22	2019-02-22	Standard	L				
21		What is your favorite cereal?	2019-02-22	2019-02-22	2019-02-22	2019-02-23	Standard	D				
33		Have you ever been to a zoo?	2019-02-08	2019-02-23	2019-02-23	2019-02-23	Standard	D				
30		If you had to swap lives with someone famous, who would you choose?	2019-02-07	2019-02-07	2019-02-07	2019-02-24	Standard	D				
24		Where do you see yourself in ten years?	2019-02-10	2019-02-10	2019-02-10	2019-02-24	Standard	L				
35		What is your first memory?	2019-02-23	2019-02-23	2019-02-23	2019-02-25	Fixed Date	V	2019-02-26			
29		Which do you like better, TV or movies?	2019-02-23	2019-02-23	2019-02-23	2019-02-28	Standard	D				
22		What is your favorite holiday?	2019-02-27	2019-02-27	2019-02-27	2019-02-28	Standard	D				
23		Have you ever slept outside?	2019-02-27	2019-02-27	2019-02-27	2019-02-28	Standard	V				
27		What is the strangest thing you've eaten?	2019-02-27	2019-02-27	2019-02-27	2019-02-28	Standard	D				
25		What is your favorite season?	2019-02-27	2019-02-27	2019-02-28	2019-02-28	Standard	C			2019-02-28	
26		What is your favorite clothing brand?	2019-02-27	2019-02-27	2019-02-28	2019-03-02	Standard	D				
28		What is your least favorite chore?	2019-02-08	2019-02-08	2019-02-08		Standard	D				
34		When is your birthday?	2019-02-09	2019-02-09	2019-02-09		Fixed Date	D2	2019-03-06			
32		What is your favorite Disney movie?	2019-02-09	2019-02-09	2019-02-09		Fixed Date	D	2019-03-07			
42		What is your favorite dessert?	2019-02-27	2019-02-27	2019-03-02		Standard	V				
31		What is your favorite flavor of ice cream?	2019-02-22	2019-02-22			Expedite	D		13		
38		If you could have a superpower, what would you choose?	2019-02-27	2019-02-27			Fixed Date	D	2019-03-11			
36		Which do you like better, city or country?	2019-02-27	2019-02-27			Intangible	V				
37		What is your favorite cheese?	2019-02-27	2019-02-27			Standard	V				
40		Where is you favorite place to eat?	2019-02-27	2019-02-27			Standard	V				
41		Which do you like better, rain or snow?	2019-02-27	2019-02-27			Standard	D				
39		Which do you like better, beer or wine?	2019-03-05				Standard	D				
""")

In [ ]:
df = pd.read_csv(TESTDATA, sep="\t")

In [ ]:
df